In [ ]:
import json

In [ ]:
# 1. Setup Environment
dbutils.widgets.text("target_env", "dev")
env = dbutils.widgets.get("target_env")

# 2. Define the Catalog explicitly
# This matches the name in your screenshot
catalog_name = "learning_lab_catalog"

# 3. Create the Full Path (Catalog.Schema)
# Result: "learning_lab_catalog.lakebridge_recon_dev"
output_schema = f"{catalog_name}.lakebridge_recon_{env}"

print(f" Setting up environment: {env}")
print(f" Creating Schema in: {output_schema}")

# 4. Create it!
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {output_schema}")

# OPTIONAL: Set this as the current default so you don't have to type it again
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA lakebridge_recon_{env}")

In [ ]:
dbutils.widgets.text(
    "config_path",
    "/Volumes/learning_lab_catalog/recon_csv/recon_config/recon_config.csv"
)

In [ ]:
config_path = dbutils.widgets.get("config_path")

config_df = (
    spark.read
         .option("header", True)
         .csv(config_path)
)

In [ ]:
table_configs = []

for row in config_df.collect():
    table_configs.append({
        "data_source": row["data_source"],
        "report_type": row["validation"],
        "secret_scope": row["scope_name"],

        "source_catalog": row["source_catalog"],
        "source_schema": row["source_schema"],
        "source_table": row["source_table"],

        "target_catalog": row["target_catalog"],
        "target_schema": row["target_schema"],
        "target_table": row["target_table_name"],

        "join_columns": row["primary_key"].split(","),

        # optional fields (safe if blank)
        "filters": row["filters"],
        "transformation": row["transformation"],
        "column_mapping": row["column_mapping"]
    })

In [ ]:
json_output = json.dumps(table_configs)

# PUBLISH TASK VALUE FOR NEXT TASK
dbutils.jobs.taskValues.set(
    key="result",
    value=json_output
)

# OPTIONAL: still exit cleanly
dbutils.notebook.exit("OK")
